# Results

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

## Load Data

In [4]:
# Read in Parquet File
df_final = pd.read_parquet('02_Clean_Data/df_final.parquet')

# Shuffle Rows of Data Using the Same Random Seed Used in the Modeling Notebook
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

In [14]:
# Read in Testing Results
rnn_results = pd.read_csv('06_Testing_Results/RNN_01_results.csv')
lstm_01_results = pd.read_csv('06_Testing_Results/lstm_01_results.csv')
# lstm_02_results = pd.read_csv('06_Testing_Results/lstm_02_results.csv')
lstm_03_results = pd.read_csv('06_Testing_Results/lstm_03_results.csv')
lstm_04_results = pd.read_csv('06_Testing_Results/lstm_04_results.csv')
gru_results = pd.read_csv('06_Testing_Results/GRU_results.csv')


## Data Preparation